In [1]:
# Import the libraries
import torch
import torch.nn as nn
import numpy as np

Till now you have worked with numpy to create and manage arrays. However, Pytorch has its own way to define arrays, or tensors in general, that are more convenient for computational efficiency:

The torch.tensor command has the same purpose of the np.array, but in PyTorch everything is a Tensor as opposed to a vector or matrix. We define types in PyTorch using the dtype=torch.xxx command.



In [2]:
X = torch.tensor(([2, 9], [1, 5], [3, 6]), dtype=torch.float) # 3 X 2 tensor
y = torch.tensor(([92], [100], [89]), dtype=torch.float) # 3 X 1 tensor
test_sample = torch.tensor(([4, 8]), dtype=torch.float) # 1 X 2 tensor

You can inspect the size of the tensors:

In [4]:
print(X.size())
print(y.size())
print(test_sample.size())

torch.Size([3, 2])
torch.Size([3, 1])
torch.Size([2])


If you already have a numpy array that you want to convert in tensor you can use:


In [5]:
np.random.seed(42)
numpy_array = np.array(np.random.rand(1000,10))
pytorch_tensor = torch.from_numpy(numpy_array)
print(pytorch_tensor.size())

torch.Size([1000, 10])


Pytorch has the advantage to be much more customizable than Keras, but this means that you have to manually define more things. 

Indeed, a neural network in Pytorch is a subclass of the nn.Module parent class.



In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self ):
        super(NeuralNetwork, self).__init__()
        pass

The code above is how you define a neural network class using Pytorch. You can add some arguments to specify the input and the output sizes, or the number of neurons in the hidden layer:

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, num_hidden):
        super(NeuralNetwork, self).__init__()
        pass


The fully-connected layer in Pytorch (equivalent to the Dense layer in Keras) is given by the function nn.Linear() that takes as input the input shape and the number of neurons. We can define the layers as follows:

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, num_hidden):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, num_hidden)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(num_hidden, 1)

We can instantiate a neural network as we do for a class (in the following example I will pass 10 as input_dim and 5 neurons for the hidden layer):

In [9]:
net = NeuralNetwork(10, 5)


In [10]:
net

NeuralNetwork(
  (linear1): Linear(in_features=10, out_features=5, bias=True)
  (sigmoid): Sigmoid()
  (linear2): Linear(in_features=5, out_features=1, bias=True)
)

In class you have seen that there are two main steps that are repeated over and over during the training process: the forward pass and backward pass. The forward is just making your input going through the network, doing the weights multiplication and so on as you have seen for the perceptron. While in Keras you have the fit() method that does both the step for you, in Pytorch, you have to implement the methods yourself:

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, num_hidden):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, num_hidden)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(num_hidden, 1)

    def forward(self, x):
        l1 = self.linear1(x)
        activation = self.sigmoid(l1)
        l2 = self.linear2(activation)
        output = self.sigmoid(l2)
        return output

As you can see in the code, each step in the forward pass is assigned to a variable, that is then passed to next layer!

- The input x is passed in a linear layer
- the output of that is assigned to the variable l1
- l1 is activated using the sigmoid and assigned to the activation variable
- the activation variable is passed to the second layer...
...till the output that is returned!

Let's create a sample to play with and make a forward pass calling the forward method:

In [12]:
sample = torch.from_numpy(np.array(np.random.rand(10), dtype=np.float32))

In [13]:
sample.size()

torch.Size([10])

In [14]:
net = NeuralNetwork(10, 5)

In [15]:
net.forward(sample)

tensor([0.4547], grad_fn=<SigmoidBackward>)

As expected, you got a single value between 0 and 1 (the output of the sigmoid).
Now that we have implemented the forward pass, we need to implement the backpropagation! Not from scratch, don't worry!

First, we need to define both an optimizer and a loss function to use this training. I chose Adam as optimizer, since it is a common choice among the scientific community, and the Binary Cross Entropy loss since we are in the binary classification setting.

In [16]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, num_hidden):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, num_hidden)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(num_hidden, 1)

    def forward(self, x):
        l1 = self.linear1(x)
        activation = self.sigmoid(l1)
        l2 = self.linear2(activation)
        output = self.sigmoid(l2)
        return output
        

#torch_fit(x_tensor, y_true_tensor, model=model, loss=loss, lr=0.1, num_epochs=30)

loss = nn.BCELoss()
model = NeuralNetwork(10, 5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
x = np.random.rand(1000,10)
y = np.random.randint(0, 2, 1000)
x_tensor = torch.tensor(x).float()
y_true_tensor = torch.tensor(y).float()
y_true_tensor = y_true_tensor.view(1000,1) # view function is the same as reshape in numpy
y_pred_tensor = model(x_tensor)
loss_value = loss(y_pred_tensor, y_true_tensor)
print(f"Initial loss: {loss_value.item():.2f}")


Initial loss: 0.69


and then wrap the training process in a unique function:

In [17]:
def torch_fit(x, y, model, loss, lr, num_epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        y_pred_tensor = model(x_tensor)
        loss_value = loss(y_pred_tensor, y_true_tensor)
        print(f'Epoch {epoch}, loss {loss_value.item():.2f}')
        loss_value.backward()
        optimizer.step()
    return model

model = torch_fit(x_tensor, y_true_tensor, model=model, loss=loss, lr=0.1, num_epochs=100)


Epoch 0, loss 0.69
Epoch 1, loss 0.69
Epoch 2, loss 0.69
Epoch 3, loss 0.69
Epoch 4, loss 0.69
Epoch 5, loss 0.69
Epoch 6, loss 0.69
Epoch 7, loss 0.69
Epoch 8, loss 0.69
Epoch 9, loss 0.69
Epoch 10, loss 0.69
Epoch 11, loss 0.69
Epoch 12, loss 0.69
Epoch 13, loss 0.69
Epoch 14, loss 0.69
Epoch 15, loss 0.69
Epoch 16, loss 0.69
Epoch 17, loss 0.69
Epoch 18, loss 0.69
Epoch 19, loss 0.69
Epoch 20, loss 0.69
Epoch 21, loss 0.69
Epoch 22, loss 0.69
Epoch 23, loss 0.69
Epoch 24, loss 0.69
Epoch 25, loss 0.69
Epoch 26, loss 0.69
Epoch 27, loss 0.69
Epoch 28, loss 0.69
Epoch 29, loss 0.69
Epoch 30, loss 0.69
Epoch 31, loss 0.69
Epoch 32, loss 0.69
Epoch 33, loss 0.69
Epoch 34, loss 0.69
Epoch 35, loss 0.69
Epoch 36, loss 0.69
Epoch 37, loss 0.69
Epoch 38, loss 0.69
Epoch 39, loss 0.69
Epoch 40, loss 0.69
Epoch 41, loss 0.69
Epoch 42, loss 0.69
Epoch 43, loss 0.69
Epoch 44, loss 0.69
Epoch 45, loss 0.69
Epoch 46, loss 0.69
Epoch 47, loss 0.69
Epoch 48, loss 0.69
Epoch 49, loss 0.69
Epoch 50, 

Nice! You have trained your first model in Pytorch!



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=44def459-af0d-441f-8551-1feb101fde77' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>